# Convert LIDO XML to CSV format (Wien repository)

### Create a csv file for each subdirectory ###

Loop through subdirectories in the folder `data/wien/lido`. For the xml files in each subdirectory, create a csv file named after the corresponding subdirectory. The csv file is saved in the folder `output_csv`.

***It is not necessary to run the following cell since all the csv files are in `output_csv` already.***

In [6]:
import os
import csv
import pandas as pd
import numpy as np
from lxml import etree
import requests
import json

# Directory containing the XML files
folder_path = "../data/wien/lido"
namespace = {"lido": "http://www.lido-schema.org"}

# Loop through each XML file in the folder
for root, dirs, _ in os.walk(folder_path):
    artist_info_dict = {}
    for dir in dirs:
        # Initialize a list to store all record data
        data = []

        # Output CSV file path
        output_csv = "../output_csv/" + dir + ".csv"

        subfolder_path = folder_path + "/" + dir
        
        for filename in os.listdir(subfolder_path):
            if filename.endswith(".xml"):
                filepath = os.path.join(subfolder_path, filename)
                tree = etree.parse(filepath)

                # Fields
                lido_record_id = tree.xpath("string(//lido:lidoRecID)", namespaces=namespace)
                image_urls = tree.xpath("//lido:linkResource/text()", namespaces=namespace)
                types = tree.xpath("//lido:objectWorkType/lido:term/text()", namespaces=namespace)
                materials = tree.xpath("//lido:termMaterialsTech/lido:term/text()", namespaces=namespace)
                display_date = tree.xpath("string(//lido:eventDate/lido:displayDate)", namespaces=namespace)
                earliest_date = tree.xpath("string(//lido:eventDate/lido:date/lido:earliestDate)", namespaces=namespace)
                latest_date = tree.xpath("string(//lido:eventDate/lido:date/lido:latestDate)", namespaces=namespace)
                subjects = tree.xpath("//lido:subjectConcept/lido:term/text()", namespaces=namespace)
                artist_name = tree.xpath("string(//lido:nameActorSet/lido:appellationValue[@lido:pref='preferred'])", namespaces=namespace)
                actor_gnd_id = tree.xpath("string(//lido:actorID[@lido:pref='preferred'])", namespaces=namespace)
                gender = None
                nationality = None
                role = tree.xpath("string(//lido:roleActor/lido:term)", namespaces=namespace)
                birth = tree.xpath("string(//lido:vitalDatesActor/lido:earliestDate)", namespaces=namespace)
                death = tree.xpath("string(//lido:vitalDatesActor/lido:latestDate)", namespaces=namespace)
                location = tree.xpath("string(//lido:repositoryLocation/lido:namePlaceSet/lido:appellationValue)", namespaces=namespace)
                classifications = tree.xpath("//lido:classification/lido:term/text()", namespaces=namespace)
                titles = tree.xpath("//lido:titleSet/lido:appellationValue/text()", namespaces=namespace)
                rights_statements = tree.xpath("//lido:rightsResource/lido:rightsType/lido:conceptID[@lido:type='http://terminology.lido-schema.org/lido00099']/text()", namespaces=namespace)
                work_id = tree.xpath("string(//lido:workID)", namespaces=namespace)
                repository_name = tree.xpath("//lido:legalBodyName/lido:appellationValue/text()", namespaces=namespace)
                record_id = tree.xpath("string(//lido:recordID)", namespaces=namespace)
                record_links = tree.xpath("//lido:recordInfoLink/text()", namespaces=namespace)
                event_type = tree.xpath("string(//lido:eventType/lido:term)", namespaces=namespace)
                display_materials = tree.xpath("//lido:displayMaterialsTech/text()", namespaces=namespace)
                inscriptions = tree.xpath("//lido:inscriptionDescription/lido:descriptiveNoteValue/text()", namespaces=namespace)
                related_work_notes = tree.xpath("//lido:objectNote/text()", namespaces=namespace)


                if actor_gnd_id != "":
                    actor_gnd_id = actor_gnd_id.split("/")[-1]

                    if (actor_gnd_id in artist_info_dict):
                        gender = artist_info_dict[actor_gnd_id]["gender"]
                        nationality = artist_info_dict[actor_gnd_id]["nationality"]
                    else:
                        url = f"https://hub.culturegraph.org/entityfacts/{actor_gnd_id}"
                        response = requests.get(url)

                        if response.status_code == 200:
                            gnd_data = response.json()
                            gender = gnd_data.get("gender", {}).get("label", "NaN")
                            nationality_list = gnd_data.get("associatedCountry", [])
                            nationality = "; ".join([n.get("preferredName", "") for n in nationality_list]) if nationality_list else "NaN"

                            artist_info_dict[actor_gnd_id] = {"gender": gender, "nationality": nationality}
                        else:
                            print("Failed to fetch GND data:", response.status_code)



                data.append({
                    "lidoRecordId": lido_record_id,
                    "imageUrl": "; ".join(image_urls),
                    "type": "; ".join(types),
                    "material": "; ".join(materials),
                    "displayDate": display_date,
                    "earliestDate": earliest_date,
                    "latestDate": latest_date,
                    "subject": "; ".join(subjects),
                    "artistName (preferred)": artist_name,
                    "artistGNDId": actor_gnd_id,
                    "genderOfArtist": gender,
                    "nationalityOfArtist": nationality,
                    "role": role,
                    "birth": birth,
                    "death": death,
                    "location": location,
                    "title": "; ".join(titles),
                    "classification": "; ".join(classifications),
                    "rightsStatement": "; ".join(rights_statements),
                    "workID": work_id,
                    "repositoryName": repository_name,
                    "recordID": record_id,
                    "recordLinks": record_links,
                    "eventType": event_type,
                    "displayMaterialsTech": display_materials,
                    "inscriptions": inscriptions,
                    "relatedWorkNotes": related_work_notes
                })


        # For each subdirectory, convert to DataFrame and save to CSV
        df = pd.DataFrame(data)
        df.replace("", np.nan, inplace=True)
        df.to_csv(output_csv, index=False, encoding="utf-8")

        print(f"Converted xml files in {dir} to {output_csv} ({len(data)} rows)")


Failed to fetch GND data: 404


C:\Users\user\AppData\Local\Temp\ipykernel_16312\983447448.py:115: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("", np.nan, inplace=True)


Converted xml files in 00001 to ../output_csv/00001.csv (1000 rows)
Converted xml files in 00002 to ../output_csv/00002.csv (1000 rows)


KeyboardInterrupt: 

### Combine all csv files into one ###

Combine all 61 csv files into a single one named `all_wien_data.csv`

***It is not necessary to run the following cell since `all_wien_data.csv` is in `output_csv` already.***

In [ ]:
import pandas as pd
import glob
import os

# merging the files
joined_files = os.path.join("../output_csv", "000*.csv")

# A list of all joined files is returned
joined_list = glob.glob(joined_files)

# Finally, the files are joined
df_all = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)
df_all.to_csv("../output_csv/all_wien_data.csv", index=False, encoding="utf-8")

print(f'The CSV file contains {df_all.shape[0]} rows and {df_all.shape[1]} columns.')

The CSV file contains 60765 rows and 25 columns.


### Display first 20 rows of the csv file ###

In [ ]:
import pandas as pd

all_data = "../output_csv/all_wien_data.csv"

# Load and display the CSV as a table
df = pd.read_csv(all_data)
df.head(20)

,lidoRecordId,imageUrl,type,material,displayDate,earliestDate,latestDate,subject,artistName (preferred),nationality,...,classification,rightsStatement,workID,repositoryName,recordID,recordLinks,eventType,displayMaterialsTech,inscriptions,relatedWorkNotes
0,gnd2012512-4/lido/tms_10026,http://sammlungenonline.albertina.at/cc/imagep...,Zeichnung; Zeichnung,Feder; Feder,NaN,1614.0,1614.0,NaN,"Braun, Augustin",NaN,...,NaN,https://creativecommons.org/publicdomain/mark/...,3276,"['Albertina', 'Wien, Albertina', 'Albertina']",tms_10026,['http://sammlungenonline.albertina.at/?query=...,Production,"['Tusche, Feder, laviert']","['M.u. ""Anno 1614. AB""']",[]
1,gnd2012512-4/lido/tms_10109,http://sammlungenonline.albertina.at/cc/imagep...,Zeichnung; Zeichnung,NaN,NaN,1716.0,1761.0,NaN,"Querfurt, August",NaN,...,NaN,https://creativecommons.org/publicdomain/mark/...,13379,"['Albertina', 'Wien, Albertina', 'Albertina']",tms_10109,['http://sammlungenonline.albertina.at/?query=...,Production,"['Rötel, Feder in Braun mit Graulavierungen']",['l. u. Herzog Albert von Sachsen-Teschen (Lug...,[]
2,gnd2012512-4/lido/tms_10150,http://sammlungenonline.albertina.at/cc/imagep...,Zeichnung; Zeichnung,Feder; Feder; laviert; laviert,NaN,1719.0,1741.0,NaN,"Sperling, Catharina",NaN,...,NaN,https://creativecommons.org/publicdomain/mark/...,4281,"['Albertina', 'Wien, Albertina', 'Albertina']",tms_10150,['http://sammlungenonline.albertina.at/?query=...,Production,"['Feder in Grau, laviert']",['l. u. Herzog Albert von Sachsen-Teschen (Lug...,[]
3,gnd2012512-4/lido/tms_101610,http://sammlungenonline.albertina.at/cc/imagep...,Druckgraphik; Druckgraphik,Holzschnitt; Holzschnitt (Druckverfahren),NaN,1506.0,1554.0,NaN,"Cranach, Lucas, der Ältere",NaN,...,NaN,https://creativecommons.org/publicdomain/mark/...,DG1929/106,"['Albertina', 'Wien, Albertina', 'Albertina']",tms_101610,['http://sammlungenonline.albertina.at/?query=...,Expression Creation,['Holzschnitt'],"['l.o. ""L C"" mit Schlangensignet']",[]
4,gnd2012512-4/lido/tms_101657,http://sammlungenonline.albertina.at/cc/imagep...,Druckgraphik; Druckgraphik,Holzschnitt; Holzschnitt (Druckverfahren),NaN,1506.0,1506.0,NaN,"Cranach, Lucas, der Ältere",NaN,...,NaN,https://creativecommons.org/publicdomain/mark/...,DG1929/111,"['Albertina', 'Wien, Albertina', 'Albertina']",tms_101657,['http://sammlungenonline.albertina.at/?query=...,Expression Creation,['Holzschnitt'],"['r.o. ""L / 1506 / C""']",[]
5,gnd2012512-4/lido/tms_101733,http://sammlungenonline.albertina.at/cc/imagep...,Druckgraphik; Druckgraphik,Radierung; Radierung (Druckverfahren),NaN,1514.0,1524.0,NaN,"Altdorfer, Albrecht",NaN,...,NaN,https://creativecommons.org/publicdomain/mark/...,DG1926/1749,"['Albertina', 'Wien, Albertina', 'Albertina']",tms_101733,['http://sammlungenonline.albertina.at/?query=...,Expression Creation,['Radierung'],"['M.u. ""AA"", in der Inschrift "".D.XIX.""']",[]
6,gnd2012512-4/lido/tms_101735,http://sammlungenonline.albertina.at/cc/imagep...,Druckgraphik; Druckgraphik,Kupferstich; Kupferstich (Druckverfahren); Kup...,NaN,1514.0,1524.0,NaN,"Altdorfer, Albrecht",NaN,...,NaN,https://creativecommons.org/publicdomain/mark/...,DG1926/1701,"['Albertina', 'Wien, Albertina', 'Albertina']",tms_101735,['http://sammlungenonline.albertina.at/?query=...,Expression Creation,['Kupferstich'],"['r.u. ""AA""']",[]
7,gnd2012512-4/lido/tms_101736,http://sammlungenonline.albertina.at/cc/imagep...,Druckgraphik; Druckgraphik,Kupferstich; Kupferstich (Druckverfahren); Kup...,NaN,1507.0,1517.0,NaN,"Altdorfer, Albrecht",NaN,...,NaN,https://creativecommons.org/publicdomain/mark/...,DG1926/1700,"['Albertina', 'Wien, Albertina', 'Albertina']",tms_101736,['http://sammlungenonline.albertina.at/?query=...,Expression Creation,['Kupferstich'],"['l.u. ""AA""']",[]
8,gnd2012512-4/lido/tms_101737,http://sammlungenonline.albertina.at/cc/imagep...,Druckgraphik; Druckgraphik,Kupferstich; Kupferstich (Druckverfahren); Kup...,NaN,1515.0,1525.0,NaN,"Altdorfer, Albrecht",NaN,...,NaN